In [1]:
import pandas as pd 

In [15]:
import re
import ast
import pyperclip
import pprint
import os
from together import Together

client = Together(api_key=open('/Users/spangher/.togetherai-usc-key.txt').read().strip())
def query_together(prompt, client=client):
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=[{
                    "role": "system",
                    "content": "You are an experienced journalist."
                },
                {
                    "role": "user",
                    "content": prompt
                }],
        max_tokens=1048,
        temperature=0.1,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
    )
    return response.choices[0].message.content

In [5]:
ls ../data/newsworthiness-training-data/

test_file.jsonl   train_file.jsonl


In [12]:
full_data_df = pd.concat([
    pd.read_json('../data/newsworthiness-training-data/train_file.jsonl', lines=True).assign(split='train'),
    pd.read_json('../data/newsworthiness-training-data/test_file.jsonl', lines=True).assign(split='test')
])

In [31]:
full_data_df['all_text']

0                              \n  Skip to main content \n 
1          \n  \n  \n \n  \n \n \n \n  \n  \n \n \n \n  ...
2          \n \n       \n    May 1, 2023 - 9:31 AM    \n...
3          \n \n       \n    May 1, 2023 - 8:27 AM    \n...
4          \n \n       \n    May 1, 2023 - 8:02 AM    \n...
                                ...                        
215210     \n  \n \n  \n  \n  \n  \n  \n  \n  \n  \n  \n...
215211     \n  \n \n  \n  \n  \n  \n  \n  \n  \n  \n  \n...
215212     \n  \n  \n  \n  \n BENJAMIN GERVIN  \n  \n On...
215213     \n  \n  \n  \n  \n  \n  \n  \n \n  \n  \n  \n...
215214     \n  \n  \n   \n  \n  YALE DAILY NEWS PUBLISHI...
Name: all_text, Length: 1073727, dtype: object

In [32]:
idx = 0
t= full_data_df['all_text'].str.strip().iloc[idx]
t

'Skip to main content'

In [36]:
PROMPT = """
    You are a helpful journalist's assistant. Here is a headline I scraped from a news website:

    ```{headline}```

    Please clean the text and COPY the exact text of just the article information. Ignoring all timestamps, HTML formatting and boilerplate text not related to the article. 
    Don't say anything else. If there is no headline information, return "None".
"""

In [34]:
p = PROMPT.format(headline=t)

In [35]:
query_together(p)

'None'

In [50]:
(full_data_df['all_text']
 .drop_duplicates().reset_index(drop=True).reset_index()
 .assign(prompt=lambda df: df['all_text'].apply(lambda x: PROMPT.format(headline=x)))
 .drop(columns='all_text')
 .to_csv('../scripts/clean_training_data/prompts_to_clean.csv.gz', compression='gzip')
)